In [1]:
import os
import re
import json
import faiss
import traceback
import huggingface_hub

from openai import OpenAI

from langchain.schema import Document
from langchain.vectorstores import FAISS 
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.docstore.in_memory import InMemoryDocstore

from langchain_openai import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain_upstage import UpstageEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

from dotenv import load_dotenv
load_dotenv("../keys.env")

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
class Args:
    retrieval_debug = False
    llm_model = "ollama"
    
    src_lang = "ko"
    if src_lang == "en":
        eval_file_path = "../dataset/eval.jsonl" ## "../dataset/en_eval.jsonl" --> 성능이 별로임.
        doc_file_path = "../dataset/en_4.0_document.jsonl" ## "../dataset/processed_documents.jsonl"
    else:
        eval_file_path = "../dataset/eval.jsonl"
        doc_file_path = "../dataset/processed_documents.jsonl"

    output_path = "./outputs/output.csv"

    ## sparse or dense or ensemble
    doc_method = "ensemble"

    ## chunking
    chunking = True
    chunk_method = "recursive" ## recursive, semantic
    semantic_chunk_method = "huggingface"
    chunk_size = 300
    chunk_overlap = 0

    ## query expension
    query_expansion = False

    ## dense
    encoder_method = "huggingface" ## huggingface, upstage, 

    ## HuggingFace
    hf_model_name = "intfloat/multilingual-e5-large-instruct"
    model_kwargs = {"device": "cuda:0"}
    encode_kwargs = {"normalize_embeddings": False,
                     "clean_up_tokenization_spaces": True}
    
    ## Upstage
    upstage_model_name = "solar-embedding-1-large-passage"
    faiss_index_file = "./index_files/upstage-faiss.npy"
    
    ## OpenAI
    openai_model_name = "text-embedding-3-large"

args = Args()

In [3]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

eval_data = load_jsonl("../dataset/eval.jsonl")
print(len(eval_data))
print(eval_data[0].keys())

doc_data = load_jsonl("../dataset/documents.jsonl")
print(len(doc_data))
print(doc_data[0].keys())

220
dict_keys(['eval_id', 'msg'])
4272
dict_keys(['docid', 'src', 'content'])


In [4]:
# 문서 로드
raw_documents = load_jsonl('../dataset/processed_documents.jsonl')

# 문서에서 docid를 포함한 Document 리스트 생성
documents = []
for doc in raw_documents:
    doc_id = doc['docid']  # JSONL 파일에서 docid 추출
    content = doc['content']  # 문서 내용 추출
    documents.append(Document(page_content=content, metadata={"doc_id": doc_id}))

In [5]:
def load_ollama_encoder(model_name):
    encoder = OllamaEmbeddings(model_name)

    return encoder

def load_upstage_encoder(model_name):
    encoder = UpstageEmbeddings(model=model_name)

    return encoder

def load_openai_encoder(model_name):
    encoder = OpenAIEmbeddings(model=model_name)

    return encoder

def load_hf_encoder(model_name, model_kwargs, encode_kwargs):
    encoder = HuggingFaceEmbeddings(model_name=model_name,
                                    model_kwargs=model_kwargs,
                                    encode_kwargs=encode_kwargs)
    
    return encoder

def load_hf_reranker(model_name, retriever):
    reranker = HuggingFaceCrossEncoder(model_name=model_name)
    compressor = CrossEncoderReranker(model=reranker, top_n=3)
    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever.as_retriever(search_kwargs={"k": 10}))

    return compression_retriever

In [6]:
def score_normalizer(val: float) -> float:
    return 1 / (1 + val)

def load_sparse_model(documents):
    from konlpy.tag import Okt
    okt = Okt()
    def tokenize(text):
        tokens = okt.morphs(text)
        return tokens

    # retriever = KiwiBM25Retriever.from_documents(documents)
    retriever = BM25Retriever.from_documents(documents, tokenizer=tokenize)
    
    return retriever

def load_dense_model(args, documents):
    if args.encoder_method == "huggingface":
        encoder = load_hf_encoder(args.hf_model_name, args.model_kwargs, args.encode_kwargs)
        print(f"Embedding Model : {args.hf_model_name}")

    elif args.encoder_method == "upstage":
        encoder = load_upstage_encoder(args.upstage_model_name)
        print(f"Embedding Model : {args.upstage_model_name}")

    elif args.encoder_method == "openai":
        encoder = load_openai_encoder(args.openai_model_name)
        print(f"Embedding Model : {args.openai_model_name}")

    index = faiss.IndexFlatL2(len(encoder.embed_query("hello world")))
    vector_store = FAISS(
        embedding_function=encoder,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        relevance_score_fn=score_normalizer
    )
    vector_store.add_documents(documents=documents)
    retriever = vector_store

    return retriever

In [7]:
sparse_retriever = load_sparse_model(documents)
sparse_retriever.k = 2

dense_retriever = load_dense_model(args, documents).as_retriever(search_kwargs={"k": 3})

ensemble_retriever1 = EnsembleRetriever(
    retrievers=[sparse_retriever, dense_retriever],  # 사용할 검색 모델의 리스트
    weights=[0.5, 0.5],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

ensemble_retriever2 = EnsembleRetriever(
    retrievers=[sparse_retriever, dense_retriever],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

ensemble_retriever3 = EnsembleRetriever(
    retrievers=[sparse_retriever, dense_retriever],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

Embedding Model : intfloat/multilingual-e5-large-instruct


In [8]:
retrievers = {
    "sparse": sparse_retriever,
    "dense": dense_retriever,
    "s5_d5_ensemble" : ensemble_retriever1,
    "s3_d7_ensemble" : ensemble_retriever2,
    "s7_d3_ensemble" : ensemble_retriever3
}

In [9]:
def print_search_results(retrievers, query):
    print(f"Query : {query}")
    for name, retriever in retrievers.items():
        results = retriever.invoke(query)
        print(f"{name} 결과:")
        for result in results:
            print(f"문서 ID: {result.metadata['doc_id']} 내용: {result.page_content}")
        print("===" * 20)

In [10]:
print_search_results(retrievers, "글리코겐의 분해는 인체에서 왜 필요한가?")

Query : 글리코겐의 분해는 인체에서 왜 필요한가?
sparse 결과:
문서 ID: 045e9913-583c-4f28-b564-698ffad62567 내용: 호흡기는 기관륜에서 갑상선의 지협과 교차됩니다. 기관륜은 인체에서 가장 중요한 기관 중 하나로, 호흡과 관련된 기능을 담당합니다. 갑상선은 또한 인체에서 중요한 역할을 수행하는데, 호르몬 분비와 대사 조절 등 다양한 기능을 담당합니다. 호흡기와 갑상선은 서로 연결되어 있으며, 기관륜에서 교차되는 것으로 알려져 있습니다. 이 교차 지점은 인체에서 호흡과 갑상선 기능이 상호작용하는 중요한 지점입니다.
문서 ID: 1f442344-084b-44f8-838b-332be289083c 내용: 근육에서의 글리코겐 분해는 초기에 포도당-1-인산염을 형성합니다. 이 과정은 근육 세포 내에서 일어나며, 에너지 생산을 위한 중요한 과정입니다. 글리코겐은 운동이나 기타 활동을 할 때 근육에서 사용되는 주요한 에너지원입니다. 글리코겐은 당분의 형태로 저장되어 있으며, 필요할 때 분해되어 포도당-1-인산염으로 변환됩니다. 이 과정은 글리코겐 분해 효소인 글리코겐 인산화 효소에 의해 촉진됩니다. 포도당-1-인산염은 근육 세포 내에서 에너지 생산에 사용되며, 근육 수축과 운동 능력을 유지하는 데 중요한 역할을 합니다. 따라서, 글리코겐 분해는 근육 세포에서 에너지 생산과 근육 기능을 유지하는 데 필수적인 과정입니다.
dense 결과:
문서 ID: 421aac6b-49ce-4697-a68f-850152f323d7 내용: 근육에서의 글리코겐 분해는 여러 가지 요인에 의해 활성화됩니다. 첫째, 운동 시 근육은 에너지를 필요로 하기 때문에 글리코겐을 분해하여 포도당으로 변환합니다. 둘째, 혈액 중의 인슐린 수준이 낮아지면 근육은 글리코겐을 분해하여 에너지를 생성합니다. 셋째, 근육 내부의 아디노신 모노인산(AMP) 수준이 증가하면 글리코겐 분해가 활성화됩니다. 따라서, 위의 어느 것도 아닌 것은 아닙니다.
문서 ID: 1f442344-0

In [11]:
retrievers = {
    "s5_d5_ensemble" : ensemble_retriever1,
    "s3_d7_ensemble" : ensemble_retriever2,
    "s7_d3_ensemble" : ensemble_retriever3
}

In [12]:
def print_search_results(retrievers, query):
    print(f"Query : {query}")
    for name, retriever in retrievers.items():
        results = retriever.invoke(query)
        print(f"{name} 결과:")
        for result in results:
            print(f"문서 ID: {result.metadata['doc_id']} 점수 : {result.metadata['score']} 내용: {result.page_content}")
        print("===" * 20)

In [13]:
print_search_results(retrievers, "글리코겐의 분해는 인체에서 왜 필요한가?")

Query : 글리코겐의 분해는 인체에서 왜 필요한가?
s5_d5_ensemble 결과:
문서 ID: 1f442344-084b-44f8-838b-332be289083c 점수 : 0.016129032258064516 내용: 근육에서의 글리코겐 분해는 초기에 포도당-1-인산염을 형성합니다. 이 과정은 근육 세포 내에서 일어나며, 에너지 생산을 위한 중요한 과정입니다. 글리코겐은 운동이나 기타 활동을 할 때 근육에서 사용되는 주요한 에너지원입니다. 글리코겐은 당분의 형태로 저장되어 있으며, 필요할 때 분해되어 포도당-1-인산염으로 변환됩니다. 이 과정은 글리코겐 분해 효소인 글리코겐 인산화 효소에 의해 촉진됩니다. 포도당-1-인산염은 근육 세포 내에서 에너지 생산에 사용되며, 근육 수축과 운동 능력을 유지하는 데 중요한 역할을 합니다. 따라서, 글리코겐 분해는 근육 세포에서 에너지 생산과 근육 기능을 유지하는 데 필수적인 과정입니다.
문서 ID: 045e9913-583c-4f28-b564-698ffad62567 점수 : 0.00819672131147541 내용: 호흡기는 기관륜에서 갑상선의 지협과 교차됩니다. 기관륜은 인체에서 가장 중요한 기관 중 하나로, 호흡과 관련된 기능을 담당합니다. 갑상선은 또한 인체에서 중요한 역할을 수행하는데, 호르몬 분비와 대사 조절 등 다양한 기능을 담당합니다. 호흡기와 갑상선은 서로 연결되어 있으며, 기관륜에서 교차되는 것으로 알려져 있습니다. 이 교차 지점은 인체에서 호흡과 갑상선 기능이 상호작용하는 중요한 지점입니다.
문서 ID: 421aac6b-49ce-4697-a68f-850152f323d7 점수 : 0.00819672131147541 내용: 근육에서의 글리코겐 분해는 여러 가지 요인에 의해 활성화됩니다. 첫째, 운동 시 근육은 에너지를 필요로 하기 때문에 글리코겐을 분해하여 포도당으로 변환합니다. 둘째, 혈액 중의 인슐린 수준이 낮아지면 근육은 글리코겐을 분해하여 에너지를 생성합니다. 셋째, 근육 내부의 아디노신 모노인산(

: 